In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_binary

import requests,lxml
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv("test.csv", header=None)

driver = webdriver.Chrome()

In [3]:
docs = []
ratings = []
reviewCounts = []
final_links = []
cnt = 0
new_df = pd.DataFrame()
with open('test.csv') as file:
    lines = file.readlines()
    for index,line in tqdm(enumerate(lines)):
        document = ''
        
        links = line.split(",")
        url = links[1].strip()
        url = url.strip("\"")
        #print(url)
        #result = requests.get(url)
        #soup = BeautifulSoup(result.content, 'lxml')
        #print(soup)
        
        driver.get(url)

        html = driver.page_source
        soup = BeautifulSoup(html)
        #print(soup)
        
        title = soup.find('h1', attrs = {'class': 'prod-ProductTitle prod-productTitle-buyBox font-bold'})
        about = soup.find('div', attrs = {'class': 'about-desc about-product-description xs-margin-top'})
        review = soup.find_all('div', attrs = {'class': 'collapsable-content-container'})
        rating = soup.find('span', itemprop = "ratingValue")
        reviewCount = soup.find('span', itemprop = "reviewCount")
        
        links = soup.find('div', attrs = {'class': 'persistent-controls'}).find("ul").findChildren("li" , recursive=False)
        #print(links)
        #
        final_link = []
        for link in links:
            var = link.find("a")
            if var is not None:
                final_link.append("https://www.walmart.com"+var['href'].split('?')[0]) 
        #print(final_link)
        #print(links)
        final_links.append(final_link)
        #print("*********************************")
        #break
        
        if title is not None:
            productTitle = title.contents
            if len(productTitle) >= 1:
                document += str(productTitle[0])

        if about is not None:
            aboutItem = about.contents
            if len(aboutItem) >= 1:
                document += ' ' + str(aboutItem[0])

        if len(review) >= 2:
            for r in review[:2]:
                rtext = r.find('div')
                document += ' ' + str(rtext.contents[0])
        
        docs.append(document)
        
        if rating is None:
            ratings.append('-1')
        else:
            ratings.append(rating.contents[0])
        if reviewCount is None:
            reviewCounts.append('-1')
        else:
            reviewCounts.append(reviewCount.contents[0])
        if(index%5==0):
            new_df["ratings"] = ratings
            new_df["reviewCounts"] = reviewCounts
            new_df["documents"] = docs
            new_df["links"] = final_links
            docs = []
            ratings = []
            reviewCounts = []
            final_links = []
            with open('test_res.csv','a',newline='') as file:
                new_df.to_csv(file, header=False,index = False)
            new_df = pd.DataFrame()
                
        
            
# df["ratings"] = ratings
# df["reviewCounts"] = reviewCounts
# df["documents"] = docs
# df["links"] = final_links

# df.to_csv("test_res.csv", index = False)

22it [00:56,  2.57s/it]


In [4]:
driver.close()